In [ ]:
import pandas as pd
from pandas import DataFrame

Limpiar el texto con spacy

In [ ]:
df = pd.read_csv('/content/picta_publicaciones_crudas.csv')
df.shape[0]


25454

In [ ]:
df = pd.read_csv('/content/picta_publicaciones_procesadas_sin_nulas.csv')
df.shape


(22882, 4)

In [ ]:
df[df.isnull().any(axis=1)]
print(df.loc[df['id'] == 6071])
print(df.loc[df['id'] == 8719])
print(df.loc[df['id'] == 5707])
print(df.loc[df['id'] == 30104])
print(df.loc[df['id'] == 15113])
print(df.loc[df['id'] == 1312])
print(df.loc[df['id'] == 2410])
print(df.loc[df['id'] == 2581])
print(df.loc[df['id'] == 2565])
print(df.loc[df['id'] == 2564])
print(df.loc[df['id'] == 2563])
print(df.loc[df['id'] == 2562])
print(df.loc[df['id'] == 1312])

In [ ]:
before = df.shape[0]
df_new = df.dropna()
after = df_new.shape[0]
delete_amount = before - after
delete_amount

0

In [ ]:
df_new.shape[0] - delete_amount
df_new.shape[0]

23150

In [ ]:
df_new['nombre'].astype(str)
df_new['nombre'].dtypes

dtype('O')

In [ ]:
!pip install -U spacy
!python -m spacy download es_core_news_lg

In [ ]:
import spacy
nlp = spacy.load('es_core_news_lg')

In [ ]:
def is_token_allowed(token: str) -> bool:
  return bool(
      token
      and not token.is_stop
      and token.is_alpha
      and not token.is_punct
  )

def preprocess_token(token: str) -> bool:
  return token.lemma_.strip().lower()


import re
import unicodedata


def filters(value):
  text = str(value)
  text = re.sub(r'[()\-_]', ' ', text)
  text = ''.join((c for c in unicodedata.normalize('NFD', value) if unicodedata.category(c) != 'Mn'))
  return text

def process_text(df, columns):
    df[columns] = df[columns].applymap(
      lambda value: str(' '.join([
        preprocess_token(token)
        for token in nlp(filters(value))
        if is_token_allowed(token)
    ])))
    return df

## Tiempo Estimado: 11 Minutos

In [ ]:
new_df = process_text(df.copy(), ['nombre', 'descripcion', 'categoria'])
new_df

,id,nombre,descripcion,categoria
0,7157,casar papel,huir millon euros fabricar nacional moneda tim...,serie
1,19699,mlb chicago cubs vs chicago white sox,probar,live
2,27272,quimica vida pedazo pan,explicar aspecto quimico biologico produccion ...,audiovisual
3,10155,concierto taino mayari,ministerio cultura cuba,audiovisual
4,10153,atalanta real madrid,comercial aplicar cuba,live
...,...,...,...,...
22877,13510,efemerid senal acn martes abril,efemerid senal acn martes abril,audiovisual
22878,13514,the war below,guerra mundial grupo minero britanico reclutar...,pelicula
22879,13513,jose marti senal acn martes abril,jose marti senal acn martes abril,audiovisual
22880,13511,chadwick boseman retrato artista,documental proponer mirada intimo arte falleci...,documental


In [ ]:
new_df.to_csv('/content/picta_publicaciones_procesadas_sin_nulas_v3.csv', index=True)

In [ ]:
df_p = pd.read_csv('/content/picta_publicaciones_procesadas.csv')

new_df_p = df_p.dropna()
new_df_p

In [ ]:
new_df_p_n = df_p[df_p.isnull().any(axis=1)]
new_df_p_n

In [ ]:
new_df_p_n.to_csv('/content/picta_publicaciones_procesadas_nulas.csv', index=False)

In [ ]:
df = df.head(1)
pd.set_option('display.max_colwidth', None)
print(df['nombre'])
print(df['descripcion'])

In [ ]:
import time

In [ ]:
# Start the timer
start_time = time.time()
new_df = process_text(df.copy(), ['nombre', 'descripcion', 'categoria'])
print(new_df['nombre'])
print(new_df['descripcion'])

# Stop the timer
end_time = time.time()

# Calculate the elapsed time in milliseconds
elapsed_time_ms = (end_time - start_time) * 1000

# Print the elapsed time
print(f"Elapsed time: {elapsed_time_ms:.2f} milliseconds")